In [1]:
%pylab inline


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import os
import numpy as np


In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, ntokens, emsize, nhead, d_hid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(emsize, dropout)
        encoder_layers = nn.TransformerEncoderLayer(emsize, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntokens, emsize)
        self.emsize = emsize
        self.decoder = nn.Linear(emsize, ntokens)
        self.ntokens = ntokens
        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, verbose=False):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.emsize)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        self.store = output.detach().numpy().copy()
        if verbose:
            print(output.shape)
        output = self.decoder(output)
        return output

In [4]:

def get_training_data(song_strings, num_songs):
    notes = list("ABCDEFGHIJ")
    chord = [[i] for i in range(10)]
    source = []
    target = []
    for s in range(num_songs):
        for i in range(40):
            sentence = [chord[notes.index(song_strings[s][(i + j) % 40])][0] for j in range(4)]
            answer = [chord[notes.index(song_strings[s][(i + j + 1) % 40])][0] for j in range(4)]
            source.append(sentence)
            target.append(answer)
    return np.array(source), np.array(target)

def predict(model, source):
    model.eval()
    source = torch.tensor(source, dtype=torch.long)
    with torch.no_grad():
        src = source.transpose(0, 1)
        output = model(src)
        predictions = output.argmax(dim=2)
    return predictions.transpose(0, 1).numpy()

def generate_predictions(model, source):
    all_predictions = []
    for i in range(80):
        current_source = np.array(source[i]).reshape(1, -1)
        current_predictions = predict(model, current_source)
        all_predictions.append(current_predictions)
    return np.array(all_predictions)

def generate_source_for_next_model(model, source):
    all_predictions = []
    source_39 = np.array(source[39]).reshape(1, -1)
    predictions_39 = predict(model, source_39)
    all_predictions.append(predictions_39)

    for i in range(39):
        current_source = np.array(source[i]).reshape(1, -1)
        current_predictions = predict(model, current_source)
        all_predictions.append(current_predictions)

    source_79 = np.array(source[79]).reshape(1, -1)
    predictions_79 = predict(model, source_79)
    all_predictions.append(predictions_79)

    for i in range(40, 79):
        current_source = np.array(source[i]).reshape(1, -1)
        current_predictions = predict(model, current_source)
        all_predictions.append(current_predictions)

    all_predictions = np.array(all_predictions)
    all_predictions = all_predictions.reshape(-1, all_predictions.shape[-1])
    return all_predictions

def calculate_accuracy(pred, target):
    pred_tensor = torch.tensor(pred.reshape(-1, pred.shape[-1]), dtype=torch.long)
    target_tensor = torch.tensor(target, dtype=torch.long)
    correct = (pred_tensor == target_tensor).sum().item()
    total = target_tensor.numel()
    return correct / total * 100

def train_model(model, source, target, num_epochs, learning_rate):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    source_tensor = torch.tensor(source, dtype=torch.long)
    target_tensor = torch.tensor(target, dtype=torch.long)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        src = source_tensor.transpose(0, 1)
        tgt = target_tensor.transpose(0, 1)
        output = model(src)
        loss = criterion(output.view(-1, model.ntokens), tgt.reshape(-1))
        loss.backward()
        optimizer.step()

    return model

In [7]:

# Parameters
ntokens = 10
emsize = 20
nhead = 4
d_hid = 20
nlayers = 2
dropout = 0.03
learning_rate = 1e-3
num_epochs = 2000
num_models = 10

song_strings = np.array([
    "ABCDEFGHIJABCDEFGHIJABCDEFGHIJABCDEFGHIJ",
    "JIHGFEDCBAJIHGFEDCBAJIHGFEDCBAJIHGFEDCBA",
])

# Initial data
source, target = get_training_data(song_strings, 2)
accuracies = []

# Training loop for 30 models
for model_idx in range(num_models):
    print(f"Training Model {model_idx + 1}")

    # Train model
    model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout)
    model = train_model(model, source, target, num_epochs, learning_rate)

    # Generate predictions and calculate accuracy
    pred = generate_predictions(model, source)
    accuracy = calculate_accuracy(pred, target)
    accuracies.append(accuracy)
    print(f"Model {model_idx + 1} Accuracy: {accuracy:.2f}%")

    # Save the model
    model_dir = f"model_{model_idx + 1}"
    os.makedirs(model_dir, exist_ok=True)
    model_save_path = os.path.join(model_dir, f'model_{model_idx + 1}.pt')
    torch.save(model.state_dict(), model_save_path)

    # Generate source for the next model
    source = generate_source_for_next_model(model, source)

# Print all accuracies
for idx, acc in enumerate(accuracies):
    print(f"Model {idx + 1} Accuracy: {acc:.2f}%")

Training Model 1
Model 1 Accuracy: 87.50%
Training Model 2
Model 2 Accuracy: 82.50%
Training Model 3
Model 3 Accuracy: 77.50%
Training Model 4
Model 4 Accuracy: 73.75%
Training Model 5
Model 5 Accuracy: 70.00%
Training Model 6
Model 6 Accuracy: 67.50%
Training Model 7
Model 7 Accuracy: 63.75%
Training Model 8
Model 8 Accuracy: 63.75%
Training Model 9
Model 9 Accuracy: 62.50%
Training Model 10
Model 10 Accuracy: 60.00%
Model 1 Accuracy: 87.50%
Model 2 Accuracy: 82.50%
Model 3 Accuracy: 77.50%
Model 4 Accuracy: 73.75%
Model 5 Accuracy: 70.00%
Model 6 Accuracy: 67.50%
Model 7 Accuracy: 63.75%
Model 8 Accuracy: 63.75%
Model 9 Accuracy: 62.50%
Model 10 Accuracy: 60.00%


In [8]:

# Parameters
ntokens = 10
emsize = 8
nhead = 2
d_hid = 8
nlayers = 2
dropout = 0.03
learning_rate = 1e-3
num_epochs = 2000
num_models = 10

song_strings = np.array([
    "ABCDEFGHIJABCDEFGHIJABCDEFGHIJABCDEFGHIJ",
    "JIHGFEDCBAJIHGFEDCBAJIHGFEDCBAJIHGFEDCBA",
])

# Initial data
source, target = get_training_data(song_strings, 2)
accuracies = []

# Training loop for 30 models
for model_idx in range(num_models):
    print(f"Training Model {model_idx + 1}")

    # Train model
    model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout)
    model = train_model(model, source, target, num_epochs, learning_rate)

    # Generate predictions and calculate accuracy
    pred = generate_predictions(model, source)
    accuracy = calculate_accuracy(pred, target)
    accuracies.append(accuracy)
    print(f"Model {model_idx + 1} Accuracy: {accuracy:.2f}%")

    # Save the model
    model_dir = f"model_{model_idx + 1}"
    os.makedirs(model_dir, exist_ok=True)
    model_save_path = os.path.join(model_dir, f'model_{model_idx + 1}.pt')
    torch.save(model.state_dict(), model_save_path)

    # Generate source for the next model
    source = generate_source_for_next_model(model, source)

# Print all accuracies
for idx, acc in enumerate(accuracies):
    print(f"Model {idx + 1} Accuracy: {acc:.2f}%")

Training Model 1
Model 1 Accuracy: 87.50%
Training Model 2
Model 2 Accuracy: 83.75%
Training Model 3
Model 3 Accuracy: 83.75%
Training Model 4
Model 4 Accuracy: 82.50%
Training Model 5
Model 5 Accuracy: 77.50%
Training Model 6
Model 6 Accuracy: 73.75%
Training Model 7
Model 7 Accuracy: 68.75%
Training Model 8
Model 8 Accuracy: 67.50%
Training Model 9
Model 9 Accuracy: 70.00%
Training Model 10
Model 10 Accuracy: 63.75%
Model 1 Accuracy: 87.50%
Model 2 Accuracy: 83.75%
Model 3 Accuracy: 83.75%
Model 4 Accuracy: 82.50%
Model 5 Accuracy: 77.50%
Model 6 Accuracy: 73.75%
Model 7 Accuracy: 68.75%
Model 8 Accuracy: 67.50%
Model 9 Accuracy: 70.00%
Model 10 Accuracy: 63.75%
